# Generate configuration using the `revisitpy` package

Source: https://revisit.dev/docs/revisitpy/

Also: 
- https://github.com/revisit-studies/revisitpy-examples
- https://github.com/revisit-studies/revisitpy

In [ ]:
import revisitpy as rvt
import pandas as pd 
import polars as pl
from datetime import date 
import json
# import revisitpy_server as rvt_server

In [2]:
# confirm working directory 

import os 
os.getcwd()

'/Users/shenglong/Downloads/study'

A revisit study has the following components in its `config.json`: [original link](https://revisit.dev/docs/typedoc/interfaces/StudyConfig/#importedlibraries)  

- `$schema`: ...
- `studyMetadata`: ...
- `uiConfig`: ...
- `importedLibraries`: ...
- `components`: ...
- `sequence`: ...

---

Some other things: 

- [`dataclasses`](https://docs.python.org/3/library/dataclasses.html): something that is related to python
- [`dataclasses.asdict`](https://docs.python.org/3/library/dataclasses.html): converts the dataclass obj to a dict

They are creating a specific dataclass obj `DataRow`: [link](https://github.com/revisit-studies/revisitpy/blob/51414e51d4c1f9c1f66b3f9c642c3c40a60138fc/src/revisitpy/revisitpy.py#L559)

```python
# Create a data class with attributes based on the headers
        DataRow = make_dataclass("DataRow", [(header, Any) for header in headers])
```

## Metadata 

- Example here: https://revisit.dev/docs/revisitpy/examples/example_jnd_study/ 

In [ ]:
# Meta Data
study_metadata = rvt.studyMetadata(
    authors=["Sheng Long"],
    organizations=["Northwestern University"],
    title='Retrieve Value Judgment Study',
    description='',
    date=date.today().strftime("%Y-%m-%d"),
    version='1.0'
)

# UI Config
ui_config = rvt.uiConfig(
  contactEmail="shenglong@u.northwestern.edu",
  logoPath="assets/revisitLogoSquare.svg",
  withSidebar=True,
  withProgressBar=False,
  nextOnEnter=True,
  minWidthSize=800,
  minHeightSize=800,
)

# print(study_metadata)
# print(ui_config)

In [ ]:
# read in external data 

ret_df = pl.read_parquet('public/vis-decode-retrieve-value/encqa_v1_ret.parquet')
# add a column for the id 
ret_df = ret_df.with_row_index("id", offset=1)
ret_df.head()

print(ret_df.select(pl.col('chart_spec')).head(1).item())

## Generate component related 

In [ ]:
# Introduction
introduction = rvt.component(type='markdown', 
                             path='vis-decode-retrieve-value/assets/introduction.md', 
                             component_name__= 'introduction')
print(introduction)
intro_seq = rvt.sequence(order='fixed', components = [introduction]) 
print(intro_seq) 

In [ ]:
# # rvt.sequence(order = 'random').from_data([(1, 2), (2, 3)])
# # asdict([1, 23])
# new_df = pd.DataFrame({"id": [i + 1 for i in range(10)]}, {"val": [i + 1 for i in range(10)]})
# print(new_df)
# new_df.to_csv('data.csv', index=False)
# print(rvt.data("data.csv"))

In [ ]:
response = rvt.response(
    id = "retrieve_value", 
    prompt = 'Your selected answer', 
    location = 'belowStimulus', 
    type = 'numerical', 
    required = True,
)

In [ ]:
n_rows = 25

new_df = pd.DataFrame({"id": [i + 1 for i in range(n_rows)]})
# print(new_df)
new_df.to_csv('data.csv', index=False)
# print(rvt.data("data.csv"))

data_sequence = rvt.sequence(order = 'random', numSamples=n_rows).from_data(rvt.data("data.csv"))
print(data_sequence)

In [ ]:
def retrieve_value_component_function(id): 
    """
    This function is used to generate the component for the retrieve value study. 
    """
    row = ret_df.filter(pl.col('id') == id)
    chart_spec_value = row.select(pl.col('chart_spec')).item()
    question = row.select(pl.col('question')).item()
    # print(json.loads(chart_spec_value))
    # print(question)
    # print(chart_spec_value)

    # get the spec for the given id 
    comp = rvt.component(
        component_name__ = f'retrieve_value_{id}',
        type = 'vega',
        response = [response],
        config = json.loads(chart_spec_value),
        instruction = f'{question}',
        instructionLocation = 'belowStimulus',
        withSidebar = False,
    )
    return comp

In [ ]:
data_sequence.component(retrieve_value_component_function)

In [ ]:
print(data_sequence)
# print(data_sequence.get_components()[0])

In [ ]:
main_sequence = rvt.sequence(order='fixed',components=[introduction]) + data_sequence

study = rvt.studyConfig(
    schema="https://raw.githubusercontent.com/revisit-studies/study/v2.3.1/src/parser/StudyConfigSchema.json",
    uiConfig=ui_config,
    studyMetadata=study_metadata,
    sequence=main_sequence,
    importedLibraries = ['virtual-chinrest']
)
print(study)

In [ ]:
# code if we are to use the rvt_server 

# process = rvt_server.serve()
# process.terminate()
# w = rvt.widget(study, server = True)

In [ ]:
str(study)
# print(study)

## Save study 

In [ ]:
# write out the study configuration 

# Write directly to file
with open('public/vis-decode-retrieve-value/config.json', 'w', encoding='utf-8') as f:
    # json.write(str(study), f, indent=2, ensure_ascii=False)
    f.write(str(study))